In [156]:
import numpy as np
from copy import copy
import pandas as pd
from tqdm.auto import tqdm

from scipy.integrate import quad
from scipy.stats import genextreme

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate

from statsmodels.discrete.discrete_model import MNLogit

## Генерация данных

In [50]:
beta_spend = np.array([1, 0.1, 0.0001, 1])
betas = np.array([[0.1, 0.000025, 0.3, 0.01],
                  [0.2, 0.000015, 0.2, 0.015],
                  [3, -0.00002, 0.5, -0.02]])

In [55]:
def generate_sample(n, betas=betas, beta_spend=beta_spend, verbose=False):
    
    ### Регрессоры
    x0 = np.ones(shape=n)
    x1 = np.exp(np.random.normal(loc=10, scale=0.7, size=n))
    x2 = np.random.poisson(lam=3, size=n)
    x2[x2 > 5] = 5
    x3 = np.round(np.random.uniform(low=20, high=100, size=n))
    x4 = np.random.poisson(lam=3, size=n)
    x4[x4 > 5] = 5
    df = pd.DataFrame(zip(x0, x1, x2, x3, x4),
                      columns=['const', 'income', 'health', 'age', 'drive'])
    
    ### Линейные индексы
    eps = genextreme.rvs(c=0, size=(n, 3))
    y_li = df[['const', 'income', 'health', 'age']] @ betas.T
    df[['y_star_Car', 'y_star_Taxi', 'y_star_Public']] = y_li + eps
    df['transport'] = np.argmax(np.array(df[['y_star_Car', 'y_star_Taxi', 'y_star_Public']]), axis=1)
    
    if verbose:
        print(df.transport.value_counts())
    
    ### Расходы
    rho = np.array([0.64, -0.25, 0.14])
    mevd, __ = quad(lambda x: genextreme.pdf(x, c=0) * x, -100, 100)
    m2evd, __ = quad(lambda x: genextreme.pdf(x, c=0) * x ** 2, -100, 100)
    vevd = m2evd - mevd ** 2
    
    vevd_rho = np.sum(vevd * rho ** 2)
    adj = np.sqrt(6) / np.pi
    sigma = 4
    
    eps_spend = sigma * adj * (eps - mevd) @ rho + \
                np.random.normal(size=n, loc=0, scale=np.sqrt(sigma ** 2 - (sigma * adj * np.sqrt(vevd_rho) ** 2)))
    
    spend_li = df[['const', 'age', 'income', 'drive']] @ beta_spend.T
    df['spend'] = spend_li + eps_spend
    df.loc[df['transport'] != 0, 'spend'] = np.nan
    
    return df

In [1]:
np.random.seed(999)

df = generate_sample(1000, verbose=True)
df.head()

NameError: name 'np' is not defined

In [67]:
df.describe()

,const,income,health,age,drive,y_star_Car,y_star_Taxi,y_star_Public,transport,spend
count,1000.0,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,280.000000
mean,1.0,28125.387539,2.826000,58.990000,2.884000,2.716238,2.609727,3.326927,1.204000,16.522477
std,0.0,22772.353569,1.513941,22.860063,1.420761,1.442548,1.367287,1.651957,0.850357,5.414768
min,1.0,1890.655466,0.000000,20.000000,0.000000,-1.104966,-0.708713,-1.303511,0.000000,1.884363
25%,1.0,13448.056133,2.000000,39.750000,2.000000,1.731258,1.679551,2.240491,0.000000,12.840438
50%,1.0,22143.597115,3.000000,59.000000,3.000000,2.537961,2.407590,3.204096,1.000000,16.417518
75%,1.0,35190.558579,4.000000,78.000000,4.000000,3.549412,3.359748,4.326264,2.000000,19.821022
max,1.0,194618.487561,5.000000,100.000000,5.000000,8.878758,8.054644,10.142650,2.000000,33.031628


In [68]:
def get_df_res(estimates, true=beta_spend, names=['beta0', 'beta1', 'beta2', 'beta3']):
    return pd.DataFrame(zip(names, true, np.mean(estimates, axis=0), 
                            np.sqrt(np.sum((estimates - true) ** 2, axis=0)), 
                            np.sum(np.abs((estimates - true) / true), axis=0) * 100),
                        columns=['Coef', 'True', 'Avg Est', 'RMSE', 'MAPE'])

### Обычная наивная регрессия

In [91]:
X, y = df[['income', 'health', 'age']], df['transport']
        
## Baseline
df_no_nans = df.dropna()
X_spend = df_no_nans[['age', 'income', 'drive']]
y_spend = df_no_nans['spend']
lm = LinearRegression().fit(X_spend, y_spend)


### Метод Дурбина-МакФаддена

In [119]:
## LogReg sklearn
# lr = LogisticRegression().fit(X, y)
# prob = lr.predict_proba(X)

## MNLogit statsmodels
lr = MNLogit(y, X).fit()
prob = lr.predict()

## Лямбды
df['lambda1'] = -np.log(prob[:, 0])
df['lambda2'] = prob[:, 1] * np.log(prob[:, 1]) / (1 - prob[:, 1])
df['lambda3'] = prob[:, 2] * np.log(prob[:, 2]) / (1 - prob[:, 2])

df_no_nans = df.dropna()
X_spend = df_no_nans[['age', 'income', 'drive']]
y_spend = df_no_nans['spend']

## Линейная модель
dmf = LinearRegression().fit(df_no_nans[['income', 'health', 'age', 
                                         'lambda1', 'lambda2', 'lambda3']], y_spend)
all_coefs.append(['Дурбин-МакФадден', dmf.intercept_] + list(dmf.coef_[:3]))

Optimization terminated successfully.
         Current function value: 0.996313
         Iterations 5


In [120]:
all_coefs = []
all_coefs.append(['Истина', 1, 0.1, 0.0001, 1])
all_coefs.append(['МНК', lm.intercept_] + list(lm.coef_))
all_coefs.append(['Дурбин-МакФадден', dmf.intercept_] + list(dmf.coef_[:3]))

pd.DataFrame(all_coefs, columns=['Метод', 'const', 'b1', 'b2', 'b3'])

,Метод,const,b1,b2,b3
0,Истина,1.000000,0.100000,0.000100,1.000000
1,МНК,6.093470,0.082754,0.000079,0.763388
2,Дурбин-МакФадден,41.180895,-0.000040,0.957561,0.093031


### Симуляция из R
Так как генерация в питоне немного не получается

In [124]:
df = pd.read_csv('/Users/markymark/Desktop/BASS/dmf_data.csv', index_col=0)

In [125]:
df.head()

,income,health,age,drive,transport,spend,lambda_1,lambda_2,lambda_3
1,14878.418921,5,27,2,Public,NaN,2.476777,-0.167808,-0.927279
2,18748.626162,2,82,5,Taxi,NaN,1.139309,-0.571882,-0.539133
3,65584.575644,3,42,2,Public,NaN,0.692606,-0.457764,-0.466095
4,23140.876404,1,64,3,Public,NaN,1.175049,-0.540462,-0.581071
5,24112.879953,5,36,0,Public,NaN,1.927762,-0.238310,-0.871698


In [126]:
np.set_printoptions(suppress=True)

X = df.dropna()[['age', 'income', 'drive']]
X_lambda = df.dropna()[['lambda_1', 'lambda_2', 'lambda_3']]
y = df.dropna().spend

LinearRegression().fit(X, y).coef_

array([0.08227652, 0.00008273, 0.95867876])

In [127]:
LinearRegression().fit(df.dropna()[['age', 'income', 'drive', 
                                    'lambda_1', 'lambda_2', 'lambda_3']], y).coef_

array([0.09110992, 0.00009568, 0.95591498, 2.2648908 , 0.36615245,
       2.60299426])

In [218]:
class BassBoost():
    
    def __init__(self, max_depth, eta=1, max_iter=5, verbose=100):
        self.trees = []
        self.max_depth = max_depth
        self.max_iter = max_iter
        self.eta = eta
        self.verbose = verbose
        
    def bias_pred(self, X, y):
        pred = np.zeros(X.shape[0])
        
        for tree in range(len(self.trees)):
            pred += tree.predict(X)
        
        return pred
    
    def __calc_grad__(self, y, pred):
        return 2 * (pred - y)
         
    def fit(self, X_index, X_bias, y):
        target = y.copy()
        
        for i in tqdm(range(self.max_iter)):
            
            linear = LinearRegression().fit(X_index, target)
            self.betas = linear.coef_
            self.intercept = linear.intercept_
            if i % self.verbose == 0:
                print(self.betas)
            grads = self.__calc_grad__(y, X_index @ self.betas + self.intercept)
            #grads = y - X_index @ self.betas - self.intercept

            tree = DecisionTreeRegressor(max_depth=self.max_depth).fit(X_bias, grads)
            self.trees.append(copy(tree))
            target -= self.eta * tree.predict(X_bias)
            
        return self.betas
        
    def predict(self, X, y, decompose=False):
        
        pass
        

In [211]:
Bass = BassBoost(max_depth=3, max_iter=1000, eta=0.7)

Bass.fit(X, X, y)

  0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: Found input variables with inconsistent numbers of samples: [2893, 1000]

In [166]:
Bass = BassBoost(max_depth=4, max_iter=10, eta=0.7)

Bass.fit(X, X_lambda, y)

  0%|          | 0/10 [00:00<?, ?it/s]

array([0.07623423, 0.00008527, 0.99074326])

### А вообще может ли он работать?

In [199]:
x1 = np.random.normal(0, 0.5, size=1000)
x2 = np.random.normal(1, 1, size=1000)
x3 = np.random.exponential(2, size=1000)
x4 = np.random.exponential(3, size=1000)

y = 2 - x1 + 3.5 * x2 - 3 * np.log(x4) ** np.round(x3)

In [200]:
np.array([x1, x2])

array([[-0.15703747, -0.87515862,  0.8209648 , ..., -0.26925996,
         0.60591385, -0.41921347],
       [ 1.77033417,  1.39559242,  2.74110077, ..., -0.31155959,
         0.75594421,  0.24301219]])

In [201]:
LinearRegression().fit(np.array([x1, x2]).T, y).coef_

array([-64.34092945,  52.10284361])

In [202]:
LinearRegression().fit(np.array([x1, x2, x3, x4]).T, y).coef_

array([ -47.04016627,   60.74618545, -181.12475285,  -53.90570988])

In [222]:
Bass = BassBoost(max_depth=10, max_iter=1000, eta=0.7, verbose=100)

Bass.fit(np.array([x1, x2]).T, np.array([x1, x2, x3, x4]).T, y)

  0%|          | 0/1000 [00:00<?, ?it/s]

[-64.34092945  52.10284361]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]
[-72.49743177  54.72006829]


array([-72.49743177,  54.72006829])

In [201]:
Bass.trees

[DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionTreeRegressor(max_depth=2, random_state=999),
 DecisionT